# BERT's ARCHITECTURE

BERT (англ. Bidirectional Encoder Representations from Transformers) — языковая модель, основанная на архитектуре трансформер, предназначенная для предобучения языковых представлений с целью их последующего применения в широком спектре задач обработки естественного языка.

Существует четыре типа предварительно обученных версий BERT в зависимости от масштаба архитектуры модели:

- BERT-Base (Cased / Un-Cased): 12-layer, 768-hidden-nodes, 12-attention-heads, 110M parameters
- BERT-Large (Cased / Un-Cased): 24-layer, 1024-hidden-nodes, 16-attention-heads, 340M parameters<

<center><img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/09/bert_encoder.png"></center>



# 1. Preprocessing

BERT опирается на Transformer (механизм внимания, который изучает контекстные связи между словами в тексте). Базовый Transformer состоит из кодировщика для чтения текстового ввода и декодера для создания прогноза для задачи. Поскольку цель BERT — создать модель представления языка, ему нужна только часть кодировщика. Входные данные для кодировщика для BERT — это последовательность токенов, которые сначала преобразуются в векторы, а затем обрабатываются в нейронной сети. Для начала, каждое вложение входных данных представляет собой комбинацию из 3 вложений:

<center><img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/09/bert_emnedding.png"></center>
Входное представление для BERT: Входные вложения представляют собой сумму вложений токенов, вложений сегментации и вложений позиции.

#2. <u>Pre-training Tasks</u></h1></center>

## 1. Masked Language Modelling</h1>

## 2. Next Sentence Prediction</h1>


Модель обучается с помощью Masked LM и Next Sentence Prediction вместе. Это делается для минимизации объединенной функции потерь двух стратегий — «вместе лучше».


# РЕАЛИЗАЦИЯ BERT

**Постановка задачи**:Имеется коллекция SMS-сообщений. Некоторые из этих сообщений являются спамом, а остальные — подлинными.

Задача — создать систему, которая автоматически определяла бы, является ли сообщение спамом или нет.

<h1 style="font-size:200%; font-family:cursive; color:navy;">1. Import Required Libraries & Dataset</h1>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast


In [ ]:
df = pd.read_csv("/content/spamdata_v2.csv")
df.head()

Набор данных состоит из двух столбцов – «label» и «text».

Столбец «text» содержит тело сообщения, а «label» – это бинарная переменная, где 1 означает спам, а 0 означает, что сообщение не является спамом.

In [ ]:
# check class distribution
df['label'].value_counts(normalize = True)

<h1 style="font-size:200%; font-family:cursive; color:navy;">2. Split the Dataset into train / test</h1>

In [ ]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['label'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

#3. Import Bert - base- uncased</h1>

**bert-base-uncased** — это предобученная языковая модель семейства BERT (Bidirectional Encoder Representations from Transformers), разработанная Google Research. Эта версия модели имеет следующие характеристики.

Основные особенности:
- Размер: Базовая версия (base), содержащая 12 слоев трансформера.
- Размеры скрытых представлений: 768 размерность.
- Количество параметров: Приблизительно 110 миллионов.
- Обучение: Обучалась на большом корпусе текста на английском языке, включая данные с Common Crawl и BooksCorpus.
- Cased vs Uncased: Модель uncased, что означает, что текст перед обработкой приводится к нижнему регистру (все буквы становятся строчными). Это упрощает обработку и уменьшает влияние различий между заглавными и строчными буквами.
  
Применение:

Эта модель используется для широкого спектра NLP-задач, таких как классификация текста, извлечение сущностей, машинный перевод, обобщение текста и другие задачи обработки естественного языка. Она доступна в библиотеках PyTorch и TensorFlow через платформы Hugging Face и других популярных фреймворков.


In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Получим длину сообщений в тренировочном наборе

In [ ]:
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins = 30)

<h1 style="font-size:200%; font-family:cursive; color:navy;">4. Tokenize & Encode the Sequences</h1>

BERT использует токенизатор WordPiece. Словарь инициализируется всеми отдельными символами языка, а затем итеративно добавляются наиболее частые/вероятные комбинации существующих слов в словаре.

Максимальная длина последовательности ввода = 512

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

<h1 style="font-size:200%; font-family:cursive; color:navy;">5. List to Tensors</h1>

In [ ]:
## convert lists to tensors

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

<h1 style="font-size:200%; font-family:cursive; color:navy;">6. Data Loader</h1>

In [ ]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

<h1 style="font-size:200%; font-family:cursive; color:navy;">7. Model Architecture</h1>

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU/CPU
device = torch.device("cpu")
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
#from transformers import AdamW
from torch.optim import Adam

# define the optimizer
optimizer = Adam(model.parameters(),lr = 1e-1)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)

print("Class Weights:",class_weights)

In [ ]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU/CPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights)

# number of training epochs
epochs = 2

<h1 style="font-size:200%; font-family:cursive; color:navy;">8. Fine - Tune (Тонкая настройка)</h1>

In [ ]:
# function to train the model
def train():

    model.train()
    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients
        model.zero_grad()

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU/CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

      # predictions are in the form of (no. of batches, size of batch, no. of classes).
      # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():

    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:

            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    #evaluate model
    valid_loss, _ = evaluate()

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<h1 style="font-size:200%; font-family:cursive; color:navy;">9. Make Predictions</h1>

In [ ]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))